In [1]:
import gym
import tensorflow as tf
import numpy as np
import random
from collections import deque
from collections import namedtuple
import torch.nn.functional as F
from torch.distributions import Categorical

import sys
import os
import argparse

from knowledge_graph import KnowledgeGraph
from kg_env import BatchKGEnvironment
from utils import *

logger = None

SavedAction = namedtuple('SavedAction', ['log_prob'])#, 'value'])

# Hyper Parameters for DQN
GAMMA = 0.9 # discount factor for target Q
INITIAL_EPSILON = 0.5 # starting value of epsilon
FINAL_EPSILON = 0.01 # final value of epsilon
REPLAY_SIZE = 10000 # experience replay buffer size
BATCH_SIZE = 32 # size of minibatch
REPLACE_TARGET_FREQ = 10 # frequency to update target Q network



In [2]:
class DQN():
    # DQN Agent
    def __init__(self, env, state_dim, act_dim, gamma=0.99,hidden_sizes=[512,256]):
    #def __init__(self, state_dim, act_dim, gamma=0.99, hidden_sizes=[512, 256]):
        # init experience replay
        self.replay_buffer = deque() #双向队列 可以从左append些什么
        # init some parameters
        self.time_step = 0
        self.epsilon = INITIAL_EPSILON
        self.state_dim = 400 #state_dim
        #self.act_dim = act_dim
        #env.observation_space.shape[0]
        '''state_dim 要改!!!'''
        self.action_dim = 32 #act_dim
        #env.action_space.n
        
        self.saved_actions = []        
        self.rewards = []
        self.entropy = []
        self.create_Q_network()
        self.create_training_method()

        # Init session
        self.session = tf.InteractiveSession()
        self.session.run(tf.global_variables_initializer())
    
    
    def create_Q_network(self):
        #print('state_dim:',self.state_dim)
        # input layer
        #???
        #self.state_input = tf.placeholder("float", [self.state_dim,None])
        self.state_input = tf.placeholder("float", [None,None,self.state_dim])
        print('state_input shape:',self.state_input.shape)
        # network weights
        with tf.variable_scope('current_net'):
            W1 = self.weight_variable([self.state_dim,20])
            b1 = self.bias_variable([20])
            W2 = self.weight_variable([20,self.action_dim])
            b2 = self.bias_variable([self.action_dim])

            # hidden layers
            
            h_layer = tf.nn.relu(tf.matmul(self.state_input,W1) + b1)
            # Q Value layer
            #print('h_layer shape:',h_layer.shape)
            #print('W2 shape:',W2.shape)
            self.Q_value = tf.matmul(h_layer,W2) + b2
            #print('Q_value shape:',self.Q_value.shape)

        with tf.variable_scope('target_net'):
            W1t = self.weight_variable([self.state_dim,20])
            b1t = self.bias_variable([20])
            W2t = self.weight_variable([20,self.action_dim])
            b2t = self.bias_variable([self.action_dim])

            # hidden layers
            h_layer_t = tf.nn.relu(tf.matmul(self.state_input,W1t) + b1t)
            # Q Value layer
            self.target_Q_value = tf.matmul(h_layer_t,W2t) + b2t

        t_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='target_net')
        e_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='current_net')

        with tf.variable_scope('soft_replacement'):
            print('t_params:',t_params)
            print('e_params:',e_params)
            self.target_replace_op = [tf.assign(t, e) for t, e in zip(t_params, e_params)]
    
    def create_training_method(self):
        self.action_input = tf.placeholder("float",[None,self.action_dim]) # one hot presentation
        #如果batch 是 32的话 这里的 none应该就要改成32
        
        self.y_input = tf.placeholder("float",[None,32])
        Q_action = tf.reduce_sum(tf.multiply(self.Q_value,self.action_input),reduction_indices = 1)
        self.cost = tf.reduce_mean(tf.square(self.y_input - Q_action))
        self.optimizer = tf.train.AdamOptimizer(0.0001).minimize(self.cost)
        self.saver = tf.train.Saver()  # 1. 初始化saver
        #optim.Adam(model.parameters(), lr=args.lr)
            
    def perceive(self,state,action,reward,next_state,done):
        one_hot_action = np.zeros(self.action_dim)
        #应该要 32*action_dim
        one_hot_action[action] = 1
        self.replay_buffer.append((state,one_hot_action,reward,next_state,done))
        #np.concatenate([user_embed, node_embed, last_node_embed, last_relation_embed, older_node_embed,
        #                           older_relation_embed])
        #
        #log_prob, value = self.saved_actions[i]
        '''--------------------------!'''
        #改成关系=1，=2，else吧
        if len(self.replay_buffer) > REPLAY_SIZE:
            self.replay_buffer.popleft()

        if len(self.replay_buffer) > BATCH_SIZE:
            self.train_Q_network()
    
    
    def train_Q_network(self):
        self.time_step += 1
        # Step 1: obtain random minibatch from replay memory
        minibatch = random.sample(self.replay_buffer,BATCH_SIZE)
        state_batch = [data[0] for data in minibatch]
        action_batch = [data[1] for data in minibatch]
        reward_batch = [data[2] for data in minibatch]
        next_state_batch = [data[3] for data in minibatch]

        # Step 2: calculate y
        y_batch = []
        print('next_state_batch:',len(next_state_batch),len(next_state_batch[0]))
        print(len(next_state_batch[0][0]))
        current_Q_batch = self.Q_value.eval(feed_dict={self.state_input: next_state_batch})
        print('current_Q_batch:',len(current_Q_batch),len(current_Q_batch[0]))
        print(len(current_Q_batch[0][0]))
        
        #eval 把值算出来
        max_action_next = np.argmax(current_Q_batch, axis=1)
        
        target_Q_batch = self.target_Q_value.eval(feed_dict={self.state_input: next_state_batch})
        #print('target_Q_batch:',len(target_Q_batch),';',len(target_Q_batch[0]),';',target_Q_batch[0])
        #a = np.append([c],[d],axis=0)
        for i in range(0,BATCH_SIZE):
            done = minibatch[i][4]
            if i == 0:
                if done:
                    y_batch = np.array(reward_batch[i])
                else:
                    target_Q_value = target_Q_batch[i, i,max_action_next[i]]
                    print(len(target_Q_batch),len(target_Q_batch[0]),len(target_Q_batch[0][0]))
                    y_batch = np.array(reward_batch[i] + GAMMA * target_Q_value)
            else:
                if done:
                    y_batch = np.append(y_batch,reward_batch[i])
                else :
                    target_Q_value = target_Q_batch[i, i,max_action_next[i]]
                    y_batch = np.append(y_batch, reward_batch[i] + GAMMA * target_Q_value)
        y_batch = y_batch.reshape(32,32)
        
        self.optimizer.run(feed_dict={self.y_input:y_batch,
                                      self.action_input:action_batch,
                                      self.state_input:state_batch})
        
        '''
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        del self.rewards[:]
        del self.saved_actions[:]
        del self.entropy[:]
        '''
        
    def egreedy_action(self,batch_state,act_mask):#batch_state = state
        
        Q_value = self.Q_value.eval(feed_dict = {self.state_input:[batch_state]})[0] #[batch_state]
        #这里的0是怎么的？
        #Q_value_state = self.target_Q_value.eval(feed_dict = {self.state_input:[batch_state]})[0]
        #print('Q_value_state:',Q_value_state)
        Q_value = torch.tensor(Q_value)
        ''''''
        Q_value[1-act_mask] = -999999.0
        Q_value = F.softmax(Q_value, dim=-1)
        #print('egreedy_action-Qvalue_act_mask:')
        #print(Q_value)
        #print('len:',Q_value.shape)
        
        
        m = Categorical(Q_value)#加起来应该不是1？说不准
        #print('m:',m)
        acts = m.sample()
        #print('egreedy acts:',acts)
        
        # [CAVEAT] If sampled action is out of action_space, choose the first action in action_space.
        '''
        acts = torch.tensor(acts)
        act_mask = torch.tensor(act_mask)
        valid_idx = act_mask.gather(1, acts.view(-1, 1)).view(-1)#torch.gather(input=act_mask,dim=1,acts.view(-1,1))
        #.view(-1行,1列)，-1 代表随便几行
        acts = np.array(acts)
        act_mask = np.array(act_mask)
        valid_idx = np.array(valid_idx)
        '''
        #acts[valid_idx == 0] = 0
        self.entropy.append(m.entropy())
        
        
        if random.random() <= self.epsilon:
            acts = []
            self.epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / 10000
            for i in range(BATCH_SIZE):
                acts.append(random.randint(0,self.action_dim - 1))
            #print('randomrandom_acts:')
            #print(acts)
            acts_log = torch.tensor(acts)
            self.saved_actions.append(SavedAction(m.log_prob(acts_log)))#, value))
            return acts #要return一行，不能只有一个值吧!!
        else:
            self.epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / 10000
            #print('notrandom_acts')
            #print(acts)
            acts_log = torch.tensor(acts)
            self.saved_actions.append(SavedAction(m.log_prob(acts_log)))#, value))
            #!!!!
            return acts #np.argmax(Q_value)
        
        '''
        self.epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / 10000
        print('notrandom_acts')
        #print(acts)
        return acts #np.argmax(Q_value)
        '''
        
        
    def action(self,batch_state,act_mask):
        Q_value = self.Q_value.eval(feed_dict = {self.state_input:[batch_state]})[0]
        Q_value = torch.tensor(Q_value)#!!!不一定对
        
        ''''''
        #print('Q_value:',Q_value,';size:',len(Q_value),';',len(Q_value[0]))
        Q_value[1-act_mask] = -999999.0
        Q_value = F.softmax(Q_value, dim=-1)

        m = Categorical(Q_value)#加起来应该不是1？说不准 
        acts = m.sample()
        # [CAVEAT] If sampled action is out of action_space, choose the first action in action_space.
        '''
        acts = torch.tensor(acts)
        act_mask = torch.tensor(act_mask)
        valid_idx = act_mask.gather(1, acts.view(-1, 1)).view(-1)#torch.gather(input=act_mask,dim=1,acts.view(-1,1))
        acts = np.array(acts)
        act_mask = np.array(act_mask)
        valid_idx = np.array(valid_idx)
        
        #.view(-1行,1列)，-1 代表随便几行
        acts[valid_idx == 0] = 0
        '''
        self.entropy.append(m.entropy())
        acts_log = torch.tensor(acts)
        self.saved_actions.append(SavedAction(m.log_prob(acts_log)))#, value))
        #!!!!
        return acts #np.argmax(Q_value)

    def update_target_q_network(self, episode):
        # update target Q netowrk
        if episode % REPLACE_TARGET_FREQ == 0:
            self.session.run(self.target_replace_op)
            self.saver.save(self.session, 'ckp')  # 2. 保存模型和变量
            #print('episode '+str(episode) +', target Q network params replaced!')
            
                
        '''
        self.saver = tf.train.Saver()  # 1. 初始化saver

        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            saver.save(sess, 'ckp')  # 2. 保存模型和变量
            !!!!!!!!!
            #with tf.Session() as sess:
                #saver = tf.import_meta_graph('ckp.meta')  # 3. 加载模型
                #saver.restore(sess, 'ckp') 
        '''

    def weight_variable(self,shape):
        initial = tf.truncated_normal(shape)
        return tf.Variable(initial)

    def bias_variable(self,shape):
        initial = tf.constant(0.01, shape = shape)
        return tf.Variable(initial)

In [3]:
class ACDataLoader(object):
    def __init__(self, uids, batch_size):
        self.uids = np.array(uids)
        self.num_users = len(uids)
        self.batch_size = batch_size
        self.reset()

    def reset(self):
        self._rand_perm = np.random.permutation(self.num_users)
        self._start_idx = 0
        self._has_next = True

    def has_next(self):
        return self._has_next


    #直接把训练集和测试机给改了就行了 序号都是 32的倍数就可
    def get_batch(self):
        if not self._has_next:
            return None
        # Multiple users per batch
        end_idx = min(self._start_idx + self.batch_size, self.num_users)
        #print('get_batch,end_idx:',end_idx,';',self.num_users)
        batch_idx = self._rand_perm[self._start_idx:end_idx]
        batch_uids = self.uids[batch_idx]
        self._has_next = self._has_next and end_idx < self.num_users
        self._start_idx = end_idx
        
        return batch_uids.tolist()

In [4]:
EPISODE = 3000 # Episode limitation
STEP = 300 # Step limitation in an episode
TEST = 5 # The number of experiment test every 100 episode

parser = argparse.ArgumentParser()
parser.add_argument('--dataset', type=str, default=BEAUTY, help='One of {clothing, cell, beauty, cd}')
parser.add_argument('--name', type=str, default='train_agent', help='directory name.')
parser.add_argument('--seed', type=int, default=123, help='random seed.')
parser.add_argument('--gpu', type=str, default='0', help='gpu device.')
parser.add_argument('--epochs', type=int, default=500, help='Max number of epochs.')
parser.add_argument('--batch_size', type=int, default=32, help='batch size.')
parser.add_argument('--lr', type=float, default=1e-4, help='learning rate.')
parser.add_argument('--max_acts', type=int, default=250, help='Max number of actions.')
#parser.add_argument('--state_dim', type=int, default=250, help='Max number of actions.')
#
parser.add_argument('--max_path_len', type=int, default=3, help='Max path length.')
parser.add_argument('--gamma', type=float, default=0.99, help='reward discount factor.')
parser.add_argument('--ent_weight', type=float, default=1e-3, help='weight factor for entropy loss')
parser.add_argument('--act_dropout', type=float, default=0.5, help='action dropout rate.')
parser.add_argument('--state_history', type=int, default=1, help='state history length')
parser.add_argument('--hidden', type=int, nargs='*', default=[512, 256], help='number of samples')
args = parser.parse_args(['--dataset',CELL])
    
    

In [5]:
os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu
args.device = torch.device('cuda:0') if torch.cuda.is_available() else 'cpu'

args.log_dir = '{}/{}'.format(TMP_DIR[args.dataset], args.name)
if not os.path.isdir(args.log_dir):
    os.makedirs(args.log_dir)

global logger
logger = get_logger(args.log_dir + '/train_log_DQN.txt')
logger.info(args)

set_random_seed(args.seed)
    


[INFO]  Namespace(act_dropout=0.5, batch_size=32, dataset='cell', device='cpu', ent_weight=0.001, epochs=500, gamma=0.99, gpu='0', hidden=[512, 256], log_dir='./tmp/Amazon_Cellphones/train_agent', lr=0.0001, max_acts=250, max_path_len=3, name='train_agent', seed=123, state_history=1)


In [6]:
def train(args):
    
    # initialize OpenAI Gym env and dqn agent
    #env = gym.make(ENV_NAME)
    env = BatchKGEnvironment(args.dataset, args.max_acts, max_path_len=args.max_path_len, state_history=args.state_history)
    uids = list(env.kg(USER).keys())
    print('uids:',len(uids))
    uids = np.arange(19488).tolist()
    agent = DQN(env,env.state_dim,env.act_dim,gamma = args.gamma,hidden_sizes = args.hidden)
    dataloader = ACDataLoader(uids, args.batch_size)
    
    #model = ActorCritic(env.state_dim, env.act_dim, gamma=args.gamma, hidden_sizes=args.hidden).to(args.device)
    #logger.info('Parameters:' + str([i[0] for i in model.named_parameters()]))

    '''    
    uids = list(env.kg(USER).keys())
    dataloader = ACDataLoader(uids, args.batch_size)
    model = ActorCritic(env.state_dim, env.act_dim, gamma=args.gamma, hidden_sizes=args.hidden).to(args.device)
    logger.info('Parameters:' + str([i[0] for i in model.named_parameters()]))
    optimizer = optim.Adam(model.parameters(), lr=args.lr)    
    '''
    episode = 1
    for epoch in range(0, args.epochs):
        ### Start epoch ###
        if epoch % 10 == 0:
            print('epoch:',epoch)
        dataloader.reset()
        while dataloader.has_next():
            batch_uids = dataloader.get_batch()
            ### Start batch episodes ###
            #print('batch_uids:',batch_uids,';',len(batch_uids))
            batch_state1 = env.reset(batch_uids)  # numpy array of [bs, state_dim]
            #print('egreedy_action, batch_state1:',batch_state1,';',len(batch_state1),';',len(batch_state1[0]))
            for step in range(STEP):#while not done:

                batch_act_mask = env.batch_action_mask(dropout=args.act_dropout)  # numpy array of size [bs, act_dim]
                '''select action'''
                #print('shape of batch state1:',batch_state1.shape)
                batch_act_idx = agent.egreedy_action(batch_state1,batch_act_mask)#batch_act_mask
                batch_state2, batch_reward, done = env.batch_step(batch_act_idx)
                batch_state2 = batch_state2.reshape((-1,400))
                agent.perceive(batch_state1,batch_act_idx,batch_reward,batch_state2,done)
                agent.rewards.append(batch_reward)
                #print('batch_state2:',batch_state2,';',len(batch_state2),';',len(batch_state2[0]))
                batch_state1 = batch_state2
                #train Q network
                if done:
                    break
                # Test every 100 episodes
            if epoch % 100 == 0:
                total_reward = 0
                for i in range(TEST):
                    #batch_uids = dataloader.get_batch()
                    #print('action batch_uids:',batch_uids)
                    ### Start batch episodes ###
                    batch_state1 = env.reset(batch_uids)  # numpy array of [bs, state_dim]
                    #state = env.reset()
                    for j in range(STEP):
                        #env.render()#在屏幕上显示画面，不需要
                        batch_act_mask = env.batch_action_mask(dropout=args.act_dropout)
                        #print('before action, batch_state1:',batch_state1,';',len(batch_state1),';',len(batch_state1[0]))
                        action = agent.action(batch_state1,batch_act_mask) # direct action for test
                        batch_state2, batch_reward, done = env.batch_step(action)
                        batch_state1 = batch_state2
                        total_reward += batch_reward
                        if done:
                            break
                ave_reward = total_reward/TEST
                if episode % 100 == 0:
                    #print ('episode: ',episode,'Evaluation Average Reward:',sum(ave_reward)/len(ave_reward))
                    logger.info('episode: ',episode,'Evaluation Average Reward:',sum(ave_reward)/len(ave_reward))
                episode = episode + 1 
        agent.update_target_q_network(epoch)

In [7]:
train(args)

Load embedding: ./tmp/Amazon_Cellphones/transe_embed.pkl
uids: 27879
state_input shape: (?, ?, 400)
t_params: [<tf.Variable 'target_net/Variable:0' shape=(400, 20) dtype=float32_ref>, <tf.Variable 'target_net/Variable_1:0' shape=(20,) dtype=float32_ref>, <tf.Variable 'target_net/Variable_2:0' shape=(20, 32) dtype=float32_ref>, <tf.Variable 'target_net/Variable_3:0' shape=(32,) dtype=float32_ref>]
e_params: [<tf.Variable 'current_net/Variable:0' shape=(400, 20) dtype=float32_ref>, <tf.Variable 'current_net/Variable_1:0' shape=(20,) dtype=float32_ref>, <tf.Variable 'current_net/Variable_2:0' shape=(20, 32) dtype=float32_ref>, <tf.Variable 'current_net/Variable_3:0' shape=(32,) dtype=float32_ref>]
epoch: 0


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:237: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:200: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


next_state_batch: 32 32
400
current_Q_batch: 32 32
32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
next_state_batch: 32 32
400
current_Q_batch: 3

next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 3

--- Logging error ---
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\logging\__init__.py", line 1034, in emit
    msg = self.format(record)
  File "C:\ProgramData\Anaconda3\lib\logging\__init__.py", line 880, in format
    return fmt.format(record)
  File "C:\ProgramData\Anaconda3\lib\logging\__init__.py", line 619, in format
    record.message = record.getMessage()
  File "C:\ProgramData\Anaconda3\lib\logging\__init__.py", line 380, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 

  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3058, in run_cell_async
    interactivity=interactivity, compiler=compiler, result=result)
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3249, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-aad596905035>", line 1, in <module>
    train(args)
  File "<ipython-input-6-0b19d1d7a6a2>", line 71, in train
    logger.info('episode: ',episode,'Evaluation Average Reward:',sum(ave_reward)/len(ave_reward))
Message: 'episode: '
Arguments: (100, 'Evaluation Average Reward:', 0.04063469469547272)


next_state_batch: 32 32
400
current_Q_batch: 32 32
32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
32 32 32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
next_state_batch: 32 32
400
current_Q_batch: 32 32
32
next_state_batch: 32 32
400


KeyboardInterrupt: 

In [7]:
train(args)

Load embedding: ./tmp/Amazon_Cellphones/transe_embed.pkl
uids: 27879
state_input shape: (?, ?, 400)
t_params: [<tf.Variable 'target_net/Variable:0' shape=(400, 20) dtype=float32_ref>, <tf.Variable 'target_net/Variable_1:0' shape=(20,) dtype=float32_ref>, <tf.Variable 'target_net/Variable_2:0' shape=(20, 32) dtype=float32_ref>, <tf.Variable 'target_net/Variable_3:0' shape=(32,) dtype=float32_ref>]
e_params: [<tf.Variable 'current_net/Variable:0' shape=(400, 20) dtype=float32_ref>, <tf.Variable 'current_net/Variable_1:0' shape=(20,) dtype=float32_ref>, <tf.Variable 'current_net/Variable_2:0' shape=(20, 32) dtype=float32_ref>, <tf.Variable 'current_net/Variable_3:0' shape=(32,) dtype=float32_ref>]
epoch: 0


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:231: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:194: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


episode:  100 Evaluation Average Reward: 0.031032263276574667
episode:  200 Evaluation Average Reward: 0.029486909729894255
episode:  300 Evaluation Average Reward: 0.06116041322020465
episode:  400 Evaluation Average Reward: 0.03670713166939095
episode:  500 Evaluation Average Reward: 0.027048354771977756
episode:  600 Evaluation Average Reward: 0.050040685496060174
epoch: 10
epoch: 20
epoch: 30
epoch: 40
epoch: 50
epoch: 60
epoch: 70
epoch: 80
epoch: 90
epoch: 100
episode:  700 Evaluation Average Reward: 0.022291885349113725
episode:  800 Evaluation Average Reward: 0.039250073107541535
episode:  900 Evaluation Average Reward: 0.035041731603269
episode:  1000 Evaluation Average Reward: 0.027887330121120612
episode:  1100 Evaluation Average Reward: 0.04729438856011256
episode:  1200 Evaluation Average Reward: 0.0332406351561076
epoch: 110
epoch: 120
epoch: 130
epoch: 140
epoch: 150
epoch: 160
epoch: 170
epoch: 180
epoch: 190
epoch: 200
episode:  1300 Evaluation Average Reward: 0.029477

In [7]:
train(args)

Load embedding: ./tmp/Amazon_Cellphones/transe_embed.pkl
uids: 27879
state_input shape: (?, ?, 400)
t_params: [<tf.Variable 'target_net/Variable:0' shape=(400, 20) dtype=float32_ref>, <tf.Variable 'target_net/Variable_1:0' shape=(20,) dtype=float32_ref>, <tf.Variable 'target_net/Variable_2:0' shape=(20, 32) dtype=float32_ref>, <tf.Variable 'target_net/Variable_3:0' shape=(32,) dtype=float32_ref>]
e_params: [<tf.Variable 'current_net/Variable:0' shape=(400, 20) dtype=float32_ref>, <tf.Variable 'current_net/Variable_1:0' shape=(20,) dtype=float32_ref>, <tf.Variable 'current_net/Variable_2:0' shape=(20, 32) dtype=float32_ref>, <tf.Variable 'current_net/Variable_3:0' shape=(32,) dtype=float32_ref>]
epoch: 0


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:233: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:196: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
3

--- Logging error ---
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\logging\__init__.py", line 1034, in emit
    msg = self.format(record)
  File "C:\ProgramData\Anaconda3\lib\logging\__init__.py", line 880, in format
    return fmt.format(record)
  File "C:\ProgramData\Anaconda3\lib\logging\__init__.py", line 619, in format
    record.message = record.getMessage()
  File "C:\ProgramData\Anaconda3\lib\logging\__init__.py", line 380, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 

  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3058, in run_cell_async
    interactivity=interactivity, compiler=compiler, result=result)
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3249, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-aad596905035>", line 1, in <module>
    train(args)
  File "<ipython-input-6-0b19d1d7a6a2>", line 71, in train
    logger.info('episode: ',episode,'Evaluation Average Reward:',sum(ave_reward)/len(ave_reward))
Message: 'episode: '
Arguments: (100, 'Evaluation Average Reward:', 0.06257213549106383)


32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
3

--- Logging error ---
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\logging\__init__.py", line 1034, in emit
    msg = self.format(record)
  File "C:\ProgramData\Anaconda3\lib\logging\__init__.py", line 880, in format
    return fmt.format(record)
  File "C:\ProgramData\Anaconda3\lib\logging\__init__.py", line 619, in format
    record.message = record.getMessage()
  File "C:\ProgramData\Anaconda3\lib\logging\__init__.py", line 380, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 

  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3058, in run_cell_async
    interactivity=interactivity, compiler=compiler, result=result)
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3249, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-aad596905035>", line 1, in <module>
    train(args)
  File "<ipython-input-6-0b19d1d7a6a2>", line 71, in train
    logger.info('episode: ',episode,'Evaluation Average Reward:',sum(ave_reward)/len(ave_reward))
Message: 'episode: '
Arguments: (200, 'Evaluation Average Reward:', 0.04947460037074052)


32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32


KeyboardInterrupt: 

In [9]:
import numpy as np
c = np.array([[1,2],[5,6]])
d = np.array([3,4])
a = np.append(c,d,axis=0)
a

ValueError: all the input arrays must have same number of dimensions

In [10]:
# ---------------------------------------------------------
# Hyper Parameters
#ENV_NAME = 'CartPole-v0'
EPISODE = 3000 # Episode limitation
STEP = 300 # Step limitation in an episode
TEST = 5 # The number of experiment test every 100 episode

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--dataset', type=str, default=BEAUTY, help='One of {clothing, cell, beauty, cd}')
    parser.add_argument('--name', type=str, default='train_agent', help='directory name.')
    parser.add_argument('--seed', type=int, default=123, help='random seed.')
    parser.add_argument('--gpu', type=str, default='0', help='gpu device.')
    parser.add_argument('--epochs', type=int, default=50, help='Max number of epochs.')
    parser.add_argument('--batch_size', type=int, default=32, help='batch size.')
    parser.add_argument('--lr', type=float, default=1e-4, help='learning rate.')
    parser.add_argument('--max_acts', type=int, default=250, help='Max number of actions.')
    #
    parser.add_argument('--max_path_len', type=int, default=3, help='Max path length.')
    parser.add_argument('--gamma', type=float, default=0.99, help='reward discount factor.')
    parser.add_argument('--ent_weight', type=float, default=1e-3, help='weight factor for entropy loss')
    parser.add_argument('--act_dropout', type=float, default=0.5, help='action dropout rate.')
    parser.add_argument('--state_history', type=int, default=1, help='state history length')
    parser.add_argument('--hidden', type=int, nargs='*', default=[512, 256], help='number of samples')
    args = parser.parse_args(['--dataset',CELL])
    
    os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu
    args.device = torch.device('cuda:0') if torch.cuda.is_available() else 'cpu'

    args.log_dir = '{}/{}'.format(TMP_DIR[args.dataset], args.name)
    if not os.path.isdir(args.log_dir):
        os.makedirs(args.log_dir)

    global logger
    logger = get_logger(args.log_dir + '/train_log.txt')
    logger.info(args)

    set_random_seed(args.seed)
    
    
    # initialize OpenAI Gym env and dqn agent
    #env = gym.make(ENV_NAME)
    env = BatchKGEnvironment(args.dataset, args.max_acts, max_path_len=args.max_path_len, state_history=args.state_history)
    agent = DQN(env)
    
    '''
    
    uids = list(env.kg(USER).keys())
    dataloader = ACDataLoader(uids, args.batch_size)
    model = ActorCritic(env.state_dim, env.act_dim, gamma=args.gamma, hidden_sizes=args.hidden).to(args.device)
    logger.info('Parameters:' + str([i[0] for i in model.named_parameters()]))
    optimizer = optim.Adam(model.parameters(), lr=args.lr)
    
    '''
    for epoch in range(1, args.epochs + 1):
        ### Start epoch ###
        dataloader.reset()
        while dataloader.has_next():
            batch_uids = dataloader.get_batch()
            ### Start batch episodes ###
            batch_state1 = env.reset(batch_uids)  # numpy array of [bs, state_dim]
            #reset返回 current_state
            #done = False
            print('done:',done)
            for step in range(STEP):#while not done:
                #没有act mask？
                batch_act_mask = env.batch_action_mask(dropout=args.act_dropout)  # numpy array of size [bs, act_dim]
                '''select action'''
                
                batch_act_idx = agent.egreedy_action(batch_state1)#batch_act_mask
                #model.select_action(batch_state, batch_act_mask, args.device)  # int
                batch_state2, batch_reward, done = env.batch_step(batch_act_idx)
                agent.perceive(batch_state1,batch_act_idx,batch_reward,batch_state2,done)
                agent.rewards.append(batch_reward)
                #train Q network
                if done:
                    break
            # Test every 100 episodes
        if epoch % 100 == 0:
            total_reward = 0
            for i in range(TEST):
                batch_uids = dataloader.get_batch()
                ### Start batch episodes ###
                batch_state1 = env.reset(batch_uids)  # numpy array of [bs, state_dim]
                #state = env.reset()
                for j in range(STEP):
                    #env.render()#在屏幕上显示画面，不需要
                    action = agent.action(batch_state1) # direct action for test
                    batch_state2, batch_reward, done = env.batch_step(action)
                    total_reward += batch_reward
                    if done:
                        break
            ave_reward = total_reward/TEST
            print ('episode: ',episode,'Evaluation Average Reward:',ave_reward)
        agent.update_target_q_network(epoch)
        
                
            ### End of episodes ###

            #lr = args.lr * max(1e-4, 1.0 - float(step) / (args.epochs * len(uids) / args.batch_size))
            #for pg in optimizer.param_groups:
                #pg['lr'] = lr

'''    
    for episode in range(EPISODE):
        # initialize task
        state = env.reset()
        # Train
        for step in range(STEP):
            action = agent.egreedy_action(state) # e-greedy action for train
            next_state,reward,done = env.batch_step(action)
            # Define reward for agent
            reward = -1 if done else 0.1
            agent.perceive(state,action,reward,next_state,done)
            #train_Q_network
            state = next_state
            if done:
                break
        # Test every 100 episodes
        if episode % 100 == 0:
            total_reward = 0
            for i in range(TEST):
                state = env.reset()
                for j in range(STEP):
                    env.render()
                    action = agent.action(state) # direct action for test
                    state,reward,done,_ = env.step(action)
                    total_reward += reward
                    if done:
                        break
            ave_reward = total_reward/TEST
            print ('episode: ',episode,'Evaluation Average Reward:',ave_reward)
        agent.update_target_q_network(episode)
'''
if __name__ == '__main__':
    main()

(1, 3, 2)
[[1 2 3]
 [4 5 6]]
(2, 3)


In [12]:
from __future__ import absolute_import, division, print_function

import os
import sys
from tqdm import tqdm
import pickle
import random
import torch
from datetime import datetime

from knowledge_graph import KnowledgeGraph
from utils import *
assert 0

AssertionError: 

In [13]:
lol = []
for j in range(32):
    for i in range(400):
        lol.append([i])

l = sum(lol, [])
len(l)

12800

In [4]:
import torch
from torch.distributions import Categorical
m = Categorical(torch.tensor([ 0.25, 0.25, 0.25, 0.25 ]))
print(m.sample())
print(m.entropy())

tensor(1)
tensor(1.3863)


In [ ]:
def train(args):
    env = BatchKGEnvironment(args.dataset, args.max_acts, max_path_len=args.max_path_len, state_history=args.state_history)
    uids = list(env.kg(USER).keys())
    dataloader = ACDataLoader(uids, args.batch_size)
    model = ActorCritic(env.state_dim, env.act_dim, gamma=args.gamma, hidden_sizes=args.hidden).to(args.device)
    logger.info('Parameters:' + str([i[0] for i in model.named_parameters()]))
    optimizer = optim.Adam(model.parameters(), lr=args.lr)

    total_losses, total_plosses, total_vlosses, total_entropy, total_rewards = [], [], [], [], []
    step = 0
    model.train()
    for epoch in range(1, args.epochs + 1):
        ### Start epoch ###
        dataloader.reset()
        while dataloader.has_next():
            batch_uids = dataloader.get_batch()
            ### Start batch episodes ###
            batch_state = env.reset(batch_uids)  # numpy array of [bs, state_dim]
            done = False
            while not done:
                batch_act_mask = env.batch_action_mask(dropout=args.act_dropout)  # numpy array of size [bs, act_dim]
                '''select action'''
                batch_act_idx = model.select_action(batch_state, batch_act_mask, args.device)  # int
                batch_state, batch_reward, done = env.batch_step(batch_act_idx)
                model.rewards.append(batch_reward)
            ### End of episodes ###

            lr = args.lr * max(1e-4, 1.0 - float(step) / (args.epochs * len(uids) / args.batch_size))
            for pg in optimizer.param_groups:
                pg['lr'] = lr

            # Update policy
            total_rewards.append(np.sum(model.rewards))
            loss, ploss, vloss, eloss = model.update(optimizer, args.device, args.ent_weight)
            total_losses.append(loss)
            total_plosses.append(ploss)
            total_vlosses.append(vloss)
            total_entropy.append(eloss)
            step += 1

            # Report performance
            if step > 0 and step % 100 == 0:
                avg_reward = np.mean(total_rewards) / args.batch_size
                avg_loss = np.mean(total_losses)
                avg_ploss = np.mean(total_plosses)
                avg_vloss = np.mean(total_vlosses)
                avg_entropy = np.mean(total_entropy)
                total_losses, total_plosses, total_vlosses, total_entropy, total_rewards = [], [], [], [], []
                logger.info(
                        'epoch/step={:d}/{:d}'.format(epoch, step) +
                        ' | loss={:.5f}'.format(avg_loss) +
                        ' | ploss={:.5f}'.format(avg_ploss) +
                        ' | vloss={:.5f}'.format(avg_vloss) +
                        ' | entropy={:.5f}'.format(avg_entropy) +
                        ' | reward={:.5f}'.format(avg_reward))
        ### END of epoch ###

        policy_file = '{}/policy_model_epoch_{}.ckpt'.format(args.log_dir, epoch)
        logger.info("Save model to " + policy_file)
        torch.save(model.state_dict(), policy_file)


